# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0  port-aux-francais -49.3500   70.2167    279.84        97   
1        1             moyale   3.5167   39.0584    292.98        86   
2        2      port mathurin -19.6833   63.4167    298.38        77   
3        3                ola  59.5833  151.2833    258.89        75   
4        4             hihifo  59.5833  151.2833    258.89        75   

   Cloudiness  Wind Speed Country        Date  
0         100       16.34      TF  1699924180  
1         100        2.64      KE  1699924180  
2           1        2.12      MU  1699924181  
3          66        5.91      RU  1699924182  
4          66        5.91      RU  1699924182

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_2 = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo =True,
                                        tiles = "EsriImagery",
                                        frame_width = 800,
                                        frame_height = 600,
                                        size = "Humidity",
                                        scale = 0.5,                                      
                                       
                                       )

# Display the map
map_plot_2

C:\Users\PC\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
#Define the criteria (e.e, max temprature, humidity, cloudiness, wind speed)
#filter the DataFrame based on the criteria
city_data_df = city_data_df[
    (city_data_df["Max Temp"] <= 300) & (city_data_df["Cloudiness"] == 40) & (city_data_df["Wind Speed"]<= 19)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

City_ID                                 City      Lat       Lng  \
217      217                        fort st. john  56.2499 -120.8529   
218      218                               howrah  22.5892   88.3103   
238      238                             qurayyat  31.3318   37.3428   
245      245                villafranca di verona  45.3524   10.8402   
320      320                               dryden  49.7833  -92.7503   
321      321                             tallulah  32.4085  -91.1868   
325      325                            pekanbaru   0.5333  101.4500   
333      333                            essaouira  31.5125   -9.7700   
385      385                                manta  -0.9500  -80.7333   
386      386  fraccionamiento arboledas san ramon  -0.9500  -80.7333   
412      412                               medina  24.4686   39.6142   

     Max Temp  Humidity  Cloudiness  Wind Speed Country        Date  
217    268.09        93          40        2.57      CA  1699924211  
218    294.15        88          40        2.57      IN  1699924256  
238    290.25        59          40        4.12      SA  1699924263  
245    282.44        93          40        1.03      IT  1699924266  
320    273.31        86          40        2.57      CA  1699924296  
321    290.18        77          40        2.06      US  1699924297  
325    298.33       100          40        2.57      ID  1699924300  
333    291.87        42          40        1.54      MA  1699924302  
385    297.03        88          40        5.66      EC  1699924320  
386    297.03        88          40        5.66      EC  1699924320  
412    298.39        29          40        1.54      SA  1699924333

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =city_data_df[["City","Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country      Lat       Lng  Humidity  \
217                        fort st. john      CA  56.2499 -120.8529        93   
218                               howrah      IN  22.5892   88.3103        88   
238                             qurayyat      SA  31.3318   37.3428        59   
245                villafranca di verona      IT  45.3524   10.8402        93   
320                               dryden      CA  49.7833  -92.7503        86   
321                             tallulah      US  32.4085  -91.1868        77   
325                            pekanbaru      ID   0.5333  101.4500       100   
333                            essaouira      MA  31.5125   -9.7700        42   
385                                manta      EC  -0.9500  -80.7333        88   
386  fraccionamiento arboledas san ramon      EC  -0.9500  -80.7333        88   
412                               medina      SA  24.4686   39.6142        29   

    Hotel Name  
217             
218             
238             
245             
320             
321             
325             
333             
385             
386             
412

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # Set to 10,000 meters
 
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params = params)
    
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df

Starting hotel search
fort st. john - nearest hotel: Fort St. John Motor Inn
howrah - nearest hotel: Sun India Guest House
qurayyat - nearest hotel: دانة الشمال
villafranca di verona - nearest hotel: Hotel San Pietro
dryden - nearest hotel: Holiday Inn Express
tallulah - nearest hotel: No hotel found
pekanbaru - nearest hotel: Hotel Radja
essaouira - nearest hotel: Essaouira Wind Palace
manta - nearest hotel: La Gaviota
fraccionamiento arboledas san ramon - nearest hotel: La Gaviota
medina - nearest hotel: The Oberoi Madina


City Country      Lat       Lng  Humidity  \
217                        fort st. john      CA  56.2499 -120.8529        93   
218                               howrah      IN  22.5892   88.3103        88   
238                             qurayyat      SA  31.3318   37.3428        59   
245                villafranca di verona      IT  45.3524   10.8402        93   
320                               dryden      CA  49.7833  -92.7503        86   
321                             tallulah      US  32.4085  -91.1868        77   
325                            pekanbaru      ID   0.5333  101.4500       100   
333                            essaouira      MA  31.5125   -9.7700        42   
385                                manta      EC  -0.9500  -80.7333        88   
386  fraccionamiento arboledas san ramon      EC  -0.9500  -80.7333        88   
412                               medina      SA  24.4686   39.6142        29   

                  Hotel Name  
217  Fort St. John Motor Inn  
218    Sun India Guest House  
238              دانة الشمال  
245         Hotel San Pietro  
320      Holiday Inn Express  
321           No hotel found  
325              Hotel Radja  
333    Essaouira Wind Palace  
385               La Gaviota  
386               La Gaviota  
412        The Oberoi Madina

In [7]:
hotel_df

City Country      Lat       Lng  Humidity  \
217                        fort st. john      CA  56.2499 -120.8529        93   
218                               howrah      IN  22.5892   88.3103        88   
238                             qurayyat      SA  31.3318   37.3428        59   
245                villafranca di verona      IT  45.3524   10.8402        93   
320                               dryden      CA  49.7833  -92.7503        86   
321                             tallulah      US  32.4085  -91.1868        77   
325                            pekanbaru      ID   0.5333  101.4500       100   
333                            essaouira      MA  31.5125   -9.7700        42   
385                                manta      EC  -0.9500  -80.7333        88   
386  fraccionamiento arboledas san ramon      EC  -0.9500  -80.7333        88   
412                               medina      SA  24.4686   39.6142        29   

                  Hotel Name  
217  Fort St. John Motor Inn  
218    Sun India Guest House  
238              دانة الشمال  
245         Hotel San Pietro  
320      Holiday Inn Express  
321           No hotel found  
325              Hotel Radja  
333    Essaouira Wind Palace  
385               La Gaviota  
386               La Gaviota  
412        The Oberoi Madina

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=['Hotel Name', 'Country']  # Add additional hover columns
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)